In [167]:
import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
import pandas as pd
import numpy as np
import string
import re
import json


with open("scraper.json", mode="r") as f:
    example_arr = json.load(f)

example_arr


liability\nOrganized\nCanadian Job Bank\n30+ days ago\nReport job'],
 ['pastry maker',
  'Additional Skills\nRequisition or order materials, equipment and supplies\nOrganize and maintain inventory\nOversee sales and merchandising of baked goods\nWork Setting\nBake shop\nSpecific Skills\nPrepare special orders\nInspect kitchen and food service areas\nEnsure that the quality of products meets established standards\nBake mixed dough and batters\nPrepare dough for pies, bread, rolls and sweet goods, batters for muffins, cookies and cakes and icings and frostings according to recipes or special customer orders\nOperate machinery\nWork Conditions and Physical Capabilities\nFast-paced environment\nWork under pressure\nRepetitive tasks\nHandling heavy loads\nPhysically demanding\nManual dexterity\nAttention to detail\nHand-eye co-ordination\nAbility to distinguish between colours\nStanding for extended periods\nTight deadlines\nOvertime required\nWork Location Information\nIn shop\nFood Specia

### We will do the following:
1. Figure out number of columns.
2. Append every row.
3. Be done!


In [168]:
def compare_two_synsets(existing_word, word_to_add):
    '''
        if words are not similar, create a new one_hot_encoding,
        if words are too similar, keep the existing word as the encoding.
    '''
    prev_existing_word = existing_word
    prev_word_to_add = word_to_add

    try:
        words_1 = wn.synsets(existing_word)
        words_2 = wn.synsets(word_to_add)
        
        for word_1 in words_1:
            for word_2 in words_2:
                similarity_score = word_1.path_similarity(word_2)
                
                ## Non-None check and similarity check
                if similarity_score and similarity_score >= 1.00:
                    return prev_existing_word 
        
        return prev_word_to_add

    except KeyError:     ## We let this go
        pass

    return prev_word_to_add

In [169]:
from nltk.stem import PorterStemmer 
ps = PorterStemmer() 

empty_frame = pd.DataFrame([], columns=["Job_Title", "Skills"]) 
used_words = [] ## old_encodings that we have used before
new_words = [] ## new_encodings for the new column
stop_words = set(stopwords.words("english"))

## Figure out one-hot encodings for title
for title, _ in example_arr:
    empty_frame["Job_Title"] = ""

    # description = re.split('\W+', description) ## Split whitespace and punctuation

    ## Remove stop_words, then stem leftover words
    # new_description = [ps.stem(word) for word in description if word not in stop_words]
    empty_frame["Skills"] = ""
    empty_frame["Short_Desc"] = ""


    for new_word in re.split('\W+', title):
        result = new_word ## default case when there are no used words. We assign new column = 1 by default
        for old_word in used_words:
            result = compare_two_synsets(old_word, new_word)
            if result == new_word:
                pass 
            else:
                print("%s and %s are similar" % (old_word, new_word))
                empty_frame[result] = "" ## similarity > 0.34. Assign 1 to existing encoding.
                break
            
        if result == new_word: ## similarity < 0.34 compared to all used words
            empty_frame[new_word] = ""
            used_words.append(result)

    
    
    
    
  

Depot and Store are similar
Office and Positions are similar
Full and FULL are similar
time and TIME are similar
Line and line are similar
cooks and cook are similar
Office and Positions are similar
Early and early are similar
Assistant and assistant are similar
Line and Course are similar


In [170]:
empty_frame.columns = sorted(empty_frame.columns.tolist())
empty_frame

,,2,6,Advisor,Appointments,Assistant,Attendant,BROWNS,Branch,Clerk,...,Worker,childhood,cooks,educator,maker,packer,pastry,time,tobacco,with


In [171]:
empty_frame.columns.tolist()

['',
 '2',
 '6',
 'Advisor',
 'Appointments',
 'Assistant',
 'Attendant',
 'BROWNS',
 'Branch',
 'Clerk',
 'Construction',
 'Coordinator',
 'Crew',
 'Deli',
 'Depot',
 'Driver',
 'Early',
 'Examiner',
 'FT',
 'File',
 'Front',
 'Full',
 'Golf',
 'H',
 'Hiring',
 'Hosts',
 'Job_Title',
 'Kitchen',
 'Labourers',
 'Line',
 'Maintenance',
 'Management',
 'Manager',
 'Member',
 'Merch',
 'Morning',
 'Office',
 'Online',
 'Operator',
 'Order',
 'PT',
 'Packager',
 'Parks',
 'Patient',
 'Picker',
 'Police',
 'Production',
 'Program',
 'Recycling',
 'Retail',
 'Retailer',
 'Reviewer',
 'SCHOOLHOUSE',
 'SOCIALHOUSE',
 'Screener',
 'Seasonal',
 'Services',
 'Short_Desc',
 'Skills',
 'Sport',
 'Technician',
 'Temporary',
 'Tire',
 'Typist',
 'Utility',
 'Warehouse',
 'Worker',
 'childhood',
 'cooks',
 'educator',
 'maker',
 'packer',
 'pastry',
 'time',
 'tobacco',
 'with']

In [172]:
columns_indices = {column: i for i, column in enumerate(empty_frame.columns.tolist())}
columns_indices

{'': 0,
 '2': 1,
 '6': 2,
 'Advisor': 3,
 'Appointments': 4,
 'Assistant': 5,
 'Attendant': 6,
 'BROWNS': 7,
 'Branch': 8,
 'Clerk': 9,
 'Construction': 10,
 'Coordinator': 11,
 'Crew': 12,
 'Deli': 13,
 'Depot': 14,
 'Driver': 15,
 'Early': 16,
 'Examiner': 17,
 'FT': 18,
 'File': 19,
 'Front': 20,
 'Full': 21,
 'Golf': 22,
 'H': 23,
 'Hiring': 24,
 'Hosts': 25,
 'Job_Title': 26,
 'Kitchen': 27,
 'Labourers': 28,
 'Line': 29,
 'Maintenance': 30,
 'Management': 31,
 'Manager': 32,
 'Member': 33,
 'Merch': 34,
 'Morning': 35,
 'Office': 36,
 'Online': 37,
 'Operator': 38,
 'Order': 39,
 'PT': 40,
 'Packager': 41,
 'Parks': 42,
 'Patient': 43,
 'Picker': 44,
 'Police': 45,
 'Production': 46,
 'Program': 47,
 'Recycling': 48,
 'Retail': 49,
 'Retailer': 50,
 'Reviewer': 51,
 'SCHOOLHOUSE': 52,
 'SOCIALHOUSE': 53,
 'Screener': 54,
 'Seasonal': 55,
 'Services': 56,
 'Short_Desc': 57,
 'Skills': 58,
 'Sport': 59,
 'Technician': 60,
 'Temporary': 61,
 'Tire': 62,
 'Typist': 63,
 'Utility': 64

In [173]:
from nltk.stem import PorterStemmer 
ps = PorterStemmer() 

stop_words = set(stopwords.words("english"))

final_data = []
## Figure out one-hot encodings for title
for title, description in example_arr:
    new_row = [0 for i in range(len(empty_frame.columns))]

    new_row[columns_indices["Job_Title"]] = title
    new_row[columns_indices["Short_Desc"]] = description[0:100]

    description = re.split('\W+', description) ## Split whitespace and punctuation

    ## Remove stop_words, then stem leftover words
    new_description = [word for word in description if word not in stop_words]
    new_row[columns_indices["Skills"]] = ",".join(new_description)


    for new_word in re.split('\W+', title):
        ## We want to treat Software like software_engineer
        new_word = new_word if new_word != "Software" else "software_engineer"
        if new_word in columns_indices:
            new_row[columns_indices[new_word]] = 1

    
    final_data.append(new_row)


final_data


    
    
    
    
  


  'Our,company,provides,clients,value,unbeatable,customer,service,complete,transparency,integrity,That,makes,us,successful,We,currently,looking,service,minded,individuals,good,communication,skills,fill,several,roles,include,sales,customer,service,Daily,activities,include,Inbound,outbound,calling,setting,appointments,performing,presentations,members,unions,request,benefits,basic,computer,knowledge,completing,necessary,paperwork,quality,control,leadership,development,Position,Benefits,Extensive,training,areas,No,experience,needed,Great,benefits,Fast,paced,fun,environment,Our,reps,average,50k,80k,first,year,Leadership,development,training,Great,weekly,pay,bonuses,Note,equal,opportunity,employer,welcome,applicants,Daily,activities,include,Inbound,outbound,calling,setting,appointments,performing,presentations,members,unions,request,benefits,basic,computer,knowledge,completing,necessary,paperwork,quality,control,leadership,development,What,looking,Team,player,mentality,Punctuality,Strong,cu

In [174]:
final_df = pd.DataFrame(final_data, columns=empty_frame.columns)

In [175]:
final_df["Skills"]

0     Immediate,opening,administrative,role,company,...
1     Police,File,Reviewer,Regular,Full,Time,2,Vacan...
2     Sport,Services,Program,Attendant,Regular,Full,...
3     At,ICBC,job,make,sure,car,insurance,system,wor...
4     At,Lowe,Canada,26,000,employees,let,passion,bl...
5     At,Lowe,Canada,26,000,employees,let,passion,bl...
6     OFFICE,COORDINATOR,NOVERO,Homes,Renovations,lo...
7     Fraser,Health,continues,recognized,one,BC,Top,...
8     Do,want,work,fun,social,environment,Browns,Soc...
9     Recycling,Depot,Operator,Regular,Full,Time,One...
10    Utility,Worker,Parks,Regular,Full,Time,Parks,R...
11    We,looking,A,mature,hard,working,outgoing,pers...
12    We,small,eCommerce,company,selling,hobby,produ...
13    Urgently,hiring,Customer,Service,Storefront,Fu...
14    The,City,Burnaby,dedication,creating,sustainin...
15    We,women,health,physiotherapy,clinic,The,clini...
16    Recycling,Depot,Operator,Regular,Full,Time,One...
17    The,City,Burnaby,dedication,creating,susta

In [176]:
final_df.columns = [column.lower() for column in final_df.columns.to_list()]


In [177]:
final_df.columns

Index(['', '2', '6', 'advisor', 'appointments', 'assistant', 'attendant',
       'browns', 'branch', 'clerk', 'construction', 'coordinator', 'crew',
       'deli', 'depot', 'driver', 'early', 'examiner', 'ft', 'file', 'front',
       'full', 'golf', 'h', 'hiring', 'hosts', 'job_title', 'kitchen',
       'labourers', 'line', 'maintenance', 'management', 'manager', 'member',
       'merch', 'morning', 'office', 'online', 'operator', 'order', 'pt',
       'packager', 'parks', 'patient', 'picker', 'police', 'production',
       'program', 'recycling', 'retail', 'retailer', 'reviewer', 'schoolhouse',
       'socialhouse', 'screener', 'seasonal', 'services', 'short_desc',
       'skills', 'sport', 'technician', 'temporary', 'tire', 'typist',
       'utility', 'warehouse', 'worker', 'childhood', 'cooks', 'educator',
       'maker', 'packer', 'pastry', 'time', 'tobacco', 'with'],
      dtype='object')

In [178]:
final_df.to_csv("processScraper.csv")

In [181]:
final_df[["skills", "job_title"]].to_csv("processScraper_wo_encoding.csv")

In [166]:
final_df.columns

Index(['', '2', '6', 'Advisor', 'Appointments', 'Assistant', 'Attendant',
       'BROWNS', 'Branch', 'Clerk', 'Construction', 'Coordinator', 'Crew',
       'Deli', 'Depot', 'Driver', 'Early', 'Examiner', 'FT', 'File', 'Front',
       'Full', 'Golf', 'H', 'Hiring', 'Hosts', 'Job_Title', 'Kitchen',
       'Labourers', 'Line', 'Maintenance', 'Management', 'Manager', 'Member',
       'Merch', 'Morning', 'Office', 'Online', 'Operator', 'Order', 'PT',
       'Packager', 'Parks', 'Patient', 'Picker', 'Police', 'Production',
       'Program', 'Recycling', 'Retail', 'Retailer', 'Reviewer', 'SCHOOLHOUSE',
       'SOCIALHOUSE', 'Screener', 'Seasonal', 'Services', 'Short_Desc',
       'Skills', 'Sport', 'Technician', 'Temporary', 'Tire', 'Typist',
       'Utility', 'Warehouse', 'Worker', 'childhood', 'cooks', 'educator',
       'maker', 'packer', 'pastry', 'time', 'tobacco', 'with'],
      dtype='object')